In [9]:
from conllu import parse_incr

In [1]:
import stanza
stanza.download('el',processors='depparse')
stanza.download(lang="el",package=None,processors={"lemma":"gdt"})

2021-05-10 15:14:20 INFO: Downloading these customized packages for language: el (Greek)...
| Processor | Package |
-----------------------
| depparse  | gdt     |
| pretrain  | gdt     |

2021-05-10 15:14:21 INFO: File exists: /home/sm/stanza_resources/el/depparse/gdt.pt.
2021-05-10 15:14:21 INFO: File exists: /home/sm/stanza_resources/el/pretrain/gdt.pt.
2021-05-10 15:14:21 INFO: Finished downloading models and saved to /home/sm/stanza_resources.
2021-05-10 15:14:22 INFO: Downloading these customized packages for language: el (Greek)...
| Processor | Package |
-----------------------
| lemma     | gdt     |

2021-05-10 15:14:22 INFO: File exists: /home/sm/stanza_resources/el/lemma/gdt.pt.
2021-05-10 15:14:22 INFO: Finished downloading models and saved to /home/sm/stanza_resources.


In [104]:
# UD
nlp = stanza.Pipeline(lang='el', processors='tokenize,mwt,pos', use_gpu=False)

2021-05-10 17:52:05 INFO: Loading these models for language: el (Greek):
| Processor | Package |
-----------------------
| tokenize  | gdt     |
| mwt       | gdt     |
| pos       | gdt     |

2021-05-10 17:52:05 INFO: Use device: cpu
2021-05-10 17:52:05 INFO: Loading: tokenize
2021-05-10 17:52:05 INFO: Loading: mwt
2021-05-10 17:52:05 INFO: Loading: pos
2021-05-10 17:52:06 INFO: Done loading processors!


In [17]:
filename = "./data/ud/train.conllu"
texts = []
total_tokens = 0
with open(filename) as f:
    for sent in parse_incr(f):
        token_forms = []
        for x in sent:
            token_forms.append(x['form'])
            total_tokens +=1
        sentence = " ".join(token_forms)
        texts.append(sentence)

In [106]:
print(total_tokens)

43440


In [93]:
from tqdm.auto import tqdm
def measure(func, times=10):
    durations = [] 
    for x in tqdm(list(range(times))):
        start = time.time()
        func()
        end = time.time()
        dur = end - start
        durations.append(dur)
        
    return durations
        

In [107]:
def stanza_perf_run():
    for x in texts:
        doc = nlp(x)

In [108]:
measure(stanza_perf_run , 10)

[155.73975777626038,
 159.30799102783203,
 145.27961087226868,
 158.21757221221924,
 149.8651442527771,
 151.58325481414795,
 155.58528017997742,
 155.9983742237091,
 144.66025614738464,
 148.94389009475708]

In [113]:
stanza_times = [155.73975777626038,
 159.30799102783203,
 145.27961087226868,
 158.21757221221924,
 149.8651442527771,
 151.58325481414795,
 155.58528017997742,
 155.9983742237091,
 144.66025614738464,
 148.94389009475708]

In [114]:
from statistics import mean

In [115]:
secs = mean(stanza_times)

In [116]:
total_tokens / secs

284.81862973475836

In [53]:
from transformers import AutoTokenizer, AutoModel

In [54]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
model = AutoModel.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")


In [55]:
tokenizer.encode("Καλησπέρα με λένε Νίκο!")

[101, 8299, 351, 1284, 3310, 108, 102]

In [117]:
from examples.ud.bert_mtask.model import UDBERTMTASKModel

In [86]:
from examples.multitask.dpud.model import POSDPModel
import pickle
import torch
deprel_i2l = None
with open('deprel_i2l.pickle', 'rb') as f:
    deprel_i2l = pickle.load(f)
        
path = 'ud_features_I2L.pickle'
feat_to_I2L = None

with open(path, 'rb') as f:
    feat_to_I2L = pickle.load(f)
            
feat_to_size = {k: len(v) for k, v in feat_to_I2L.items()}


In [87]:
model_head = POSDPModel(model, deprel_i2l, feat_to_size, 0.1 )

In [121]:
model_head = UDBERTMTASKModel(model, feat_to_size , 0.1)

In [122]:
sent = "taxaxaxa"
tens = tokenizer.encode(sent , return_tensors="pt")
lens = torch.Tensor(tens.shape[1]) 
print(tens)

tensor([[  101, 28619, 12634, 12634,   182,   102]])


In [123]:
def gbertfrom_configerf_run():
    for x in texts:
        with torch.no_grad():
            tens = tokenizer.encode(x , return_tensors="pt")
            lens = torch.Tensor([tens.shape[1]]) 
            prds = model_head(tens,lens)

In [124]:
measure(gbertfrom_configerf_run , times=10)

[103.3727195262909,
 96.21718382835388,
 107.85014271736145,
 97.03432655334473,
 94.23229169845581,
 93.75064587593079,
 94.41303515434265,
 93.04832005500793,
 92.91282320022583,
 93.53021121025085]

In [125]:
durs= [103.3727195262909,
 96.21718382835388,
 107.85014271736145,
 97.03432655334473,
 94.23229169845581,
 93.75064587593079,
 94.41303515434265,
 93.04832005500793,
 92.91282320022583,
 93.53021121025085]

In [126]:
m = mean(durs)

In [127]:
total_tokens / m

449.52112659381